In [ ]:
import os  # pour utiliser les variables d'environnements

In [ ]:
import psycopg2   # Le pendant de sqlite 3 pour postgresql

# création de la base de données


In [ ]:
# Fonction qui va nous permettre de créer une Base de données.
def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        # on force la sauvegarde
        conn.set_session(autocommit=True)   
        
        # initialisation du cursor
        cursor = conn.cursor()
        
        # exécution de la requête sql de création de la base de donnée
        cursor.execute("CREATE DATABASE IF NOT EXISTS movie2")

    # gestion des erreurs    
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
    # dans tous les cas on va clôturer la session
    finally:
        cursor.close
        conn.close
    return conn
    

In [ ]:
# Pour la création de la base de donnée, nous avons du indiquer comme nom de BDD "postgre"
base_de_donnee = "postgres"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_pw')
    

In [ ]:
# appelle de la fonction de création de la BDD
ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

------------------

# Connection et création table

In [ ]:
import csv
from pathlib import Path

# Répertoire dataset (small)
repertoire_data = Path('movies')

# Chemin vers les différents fichiers de données
chemin_movies = repertoire_data / 'movies.csv'
chemin_links = repertoire_data / 'links.csv'
chemin_ratings = repertoire_data / 'ratings.csv'
chemin_tags = repertoire_data / 'tags.csv' 

----------------------------

# Gestion de la base de données

In [ ]:
# Fonction pour nous connecter à la BDD 
def ouvrir_connection(base_de_donnee, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        # création de l'objet conn via la fonction connect de psycopg2
        conn = psycopg2.connect(dbname= base_de_donnee, user=utilisateur, password = mot_passe, host="localhost", port= 5432)

        conn.set_session(autocommit=True)   # on force
        
        # initialisation du cursor
        cursor = conn.cursor()
               
    except psycopg2.Error as e:
        print("Problème lors de la connection à la base de donnée")
        print(e)
        return None
    
    finally:
        cursor.close
        conn.close
    return conn
    
# Fonction pour lire et utiliser les fichiers csv   
def lire_donnee(filename):
         
    # on ouvre une liste vide data:
    data = [] 
    # utilisation de la fonction open pour lire les fichiers csv
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        # next permet de sauter la première ligne du fichier csv (les en-têtes)
        next(reader)
        # on sauvegarde les données du fichier csv dans la liste data
        for row in reader:
            data.append(row)
                        
    return data

# fonction qui nous permet de séparer l'année du titre dans la colonne title
def couper_annee_titre_film(title):
    # on initialise deux variables qui vont récupérer respectivement le titre et l'année
    new_title=title.strip()
    year=None
    try:
        # on vérifie si le titre comporte bien une année entre parenthèses
        if new_title[-6]=='(' and new_title[-1] == ')':
            year=int(new_title[-5:-1])
            new_title=new_title[:-6].strip()
    except:
        pass
    return  new_title, year

# fonction pour supprimer une table
def supprimer_table(conn, sql_suppression_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_suppression_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la suppression de la table")
        print(e)
        return
    cursor.close()
    print("La table a été supprimée avec succès")

# fonction pour créer une table    
def creer_table(conn, sql_creer_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creer_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("La table a été crée avec succès")

# fonction qui contient une requête sql pour insérer les données    
def inserer_donnees(conn, sql_inserer_donnee, donnees):
    try:
        cursor = conn.cursor()
        # on ouvre une boucle pour parcourir les données
        for d in donnees:
            """ on appelle la fonction couper_annee_titre_film  
             qui nous renvoie une liste dont on veut la valeur
             de l'indice 1 (le titre). Données sauvegarder dans
             la variable titre_formater            
            """
            titre_formate = couper_annee_titre_film(d[1])
            # on distribue dans les variables différentes le titre et l'année
            titre = titre_formate[0]
            year = titre_formate[1]
            cursor.execute(sql_inserer_donnee, (d[0], titre, d[2], year))
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

    # fonction de vérification
def lire_table(conn, sql_lire_donnee):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_lire_donnee)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data   

In [ ]:
# SQL QUERY

sql_supprimer_table = """
    DROP TABLE IF EXISTS movie;
"""
# on crée directement une nouvelle colonne year
sql_creer_table = """
    CREATE TABLE IF NOT EXISTS movie (
    movieid int primary key,
    title text,
    genres text,
    year int
    );
"""

sql_inserer_donnee = """
    INSERT INTO movie 
    (movieid, title, genres, year)
    VALUES (%s, %s, %s, %s);
"""

sql_lire_donnee = """
    SELECT *
    FROM movie
    LIMIT 10;
"""

In [ ]:
# connection à notre nouvelle base de données movie2
base_de_donnee = "movie2"
utilisateur = "postgres"
mot_passe = os.environ.get('pg_pw')
    

In [ ]:
print(ouvrir_connection)

In [ ]:
conn = ouvrir_connection(base_de_donnee, utilisateur, mot_passe)

In [ ]:
creer_table(conn, sql_creer_table)

In [ ]:
donnees = lire_donnee(chemin_movies)

In [ ]:
inserer_donnees(conn, sql_inserer_donnee, donnees)

In [ ]:
lire_table(conn, sql_lire_donnee)